## Identifiers

In [1]:
import pickle
import sys
import spacy
from spacy import displacy
import benepar

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [4]:
from app.classes.spec.primitive import *

from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

### Testing it out

In [5]:
from app.src.primitive_identifiers.dict_set_builder import DictSetBuilder


all_dicts = DictSetBuilder.build_all()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [14]:
from app.src.primitive_extractor import PrimitiveExtractor
from app.src.primitive_identifiers.obligation_event_identifier import ObligationEventIdentifier
from app.src.primitive_identifiers.power_event_identifier import PowerEventIdentifier
from app.src.primitive_identifiers.contract_event_identifier import ContractEventIdentifier
from app.src.primitive_identifiers.event_scorer import EventScorer
from app.src.primitive_identifiers.variable_scorer import VariableScorer
from app.src.primitive_identifiers.similarity_calculator import SentenceSimilarityCalculator
from app.src.primitive_identifiers.sentence_preprocessor import SentencePreprocessor
from app.templates.meat_sale.templates import meat_sale_templates

preprocessor = SentencePreprocessor(nlp)
similarity_calculator = SentenceSimilarityCalculator(preprocessor)

# ObligationEvent
ob_event_scorer = EventScorer(all_dicts['obligation_events'])
obligation_sent_dict = meat_sale_templates['obligations']
ob_var_scorer = VariableScorer(nlp, obligation_sent_dict, similarity_calculator)
oe_identifier = ObligationEventIdentifier(ob_event_scorer, ob_var_scorer)

# PowerEvent
power_event_scorer = EventScorer(all_dicts['power_events'])
power_sent_dict = meat_sale_templates['powers']
power_var_scorer = VariableScorer(nlp, power_sent_dict, similarity_calculator)
pe_identifier = PowerEventIdentifier(power_event_scorer, power_var_scorer)

# ContractEvent
c_event_scorer = EventScorer(all_dicts['contract_events'])
c_identifier = ContractEventIdentifier(c_event_scorer)

primitive_extractor = PrimitiveExtractor([
    oe_identifier,
    pe_identifier,
    c_identifier
], 0.78)


In [15]:

test_sentence = 'seller has resumed delivery of goods'
doc = nlp(test_sentence)

results = primitive_extractor.extract(doc)

for x in results:
    print(type(x))
    # print(x)
    # p: ObligationEvent = result.primitive
    # print('event: ',p.event_name)
    # print('var:', p.obligation_variable)
    # print(result.score)

<class 'app.classes.spec.sym_event.ObligationEvent'>
